# Python - Oracle 연동

### 패키지 참조

- 사용 패키지 : cx_Oracle, sqlalchemy, pandas (+ openpyxl, xlrd)

In [1]:
import cx_Oracle as cx
from sqlalchemy import create_engine
from pandas import DataFrame, read_sql_table

### cx_Oracle을 사용한 연동
- 데이터베이스 연동 과정은 '데이터베이스 접속 -> SQL 실행 객체(cursor) 생성 -> 결과 처리 -> 데이터베이스 접속 해제'의 순서로 진행

1. 데이터베이스 접속

In [20]:
dbcon = cx.connect("hr",    # 사용자 이름
                   "hr",    # 비밀번호
                   "localhost:1521/xe")  # 데이터베이스 서버 주소

dbcon

<cx_Oracle.Connection to hr@localhost:1521/xe>

2. Cursor 객체 생성

In [21]:
cursor = dbcon.cursor()

3. 데이터 조회를 위한 SQL문 수행
   - 튜플을 원소로 갖는 리스트 형태로 조회

In [7]:
sql = "SELECT * FROM department"

try:
    cursor.execute(sql)
    result = cursor.fetchall()
    print(result)
except Exception as e:
    print("데이터 조회 실패", e)

[(101, '컴퓨터공학과', '1호관'), (102, '멀티미디어학과', '2호관'), (201, '전자공학과', '3호관'), (202, '컴퓨터공학과', '공학관'), (320, '정보통신학과', '6호관')]


- 딕셔너리를 원소로 갖는 리스트 형태로 조회
  - cx_Oracle은 별도의 딕셔너리 형태를 제공하지 않음.
  - cursor 객체의 rowfactory 프로퍼티를 직접 재정의 해야 한다.
    - lambda 식 사용

In [9]:
sql = "SELECT * FROM department"

try:
    cursor.execute(sql)

    # rowfactory 재정의 - 접속 해제 전까지 최초 1회만 수행하면 된다.
    cursor.rowfactory = lambda *args: dict(zip([d[0] for d in cursor.description], args))

    result = cursor.fetchall()
    print(result)
except Exception as e:
    print("데이터 조회 실패", e)

[{'DEPTNO': 101, 'DNAME': '컴퓨터공학과', 'LOC': '1호관'}, {'DEPTNO': 102, 'DNAME': '멀티미디어학과', 'LOC': '2호관'}, {'DEPTNO': 201, 'DNAME': '전자공학과', 'LOC': '3호관'}, {'DEPTNO': 202, 'DNAME': '컴퓨터공학과', 'LOC': '공학관'}, {'DEPTNO': 320, 'DNAME': '정보통신학과', 'LOC': '6호관'}]


4. 조회 결과를 활용한 후속 처리
    - 수업 내용 활용

In [10]:
with open('department.csv', 'w', encoding = 'utf-8') as f:
    for i, v, in enumerate(result):
        if i == 0:
            keys = list(v.keys())
            titleLine = "%s\n" % ','.join(keys)
            f.write(titleLine)
        
        values = list(v.values())
        # values의 모든 원소를 str 타입으로 변경
        values_str = list(map(str, values))

        valueLine = '%s\n' % ','.join(values_str)
        f.write(valueLine)

- 데이터프레임

In [11]:
df = DataFrame(result)
df

,DEPTNO,DNAME,LOC
0,101,컴퓨터공학과,1호관
1,102,멀티미디어학과,2호관
2,201,전자공학과,3호관
3,202,컴퓨터공학과,공학관
4,320,정보통신학과,6호관


In [12]:
df.to_csv("department2.csv", index = False, encoding = 'utf-8')

- 데이터프레임 to_csv는 open을 이용한 csv 작성보다 속도가 느려 대용량 데이터에 대해서는 open 사용 권장
  - 코드는 to_csv가 간편함

In [13]:
df.to_excel("department3.xlsx", index = False)

5. 데이터베이스 접속 해제
   - cursor 객체와 dbcon 객체를 닫아서 점유하고 있는 메모리를 컴퓨터에 반납해야 한다. (자원 반납)
   - 자원 반납은 생성된 역순으로 진행

In [22]:
try:
    cursor.close()
    dbcon.close()
except Exception as e:
    print("접속 상태가 아닙니다", e)

### sqlalchemy를 사용한 연동
- 데이터베이스의 종류를 구분하지 않고 공통된 연동 방법을 제공하는 인터페이스

1. 데이터베이스 접속하기
   - 접속 문자열 생성
     - 'oracle+cx_oracle://계정이름:비밀번호@접속주소/SID'

In [15]:
conStr = "oracle+cx_oracle://hr:hr@localhost:1521/xe"

- 접속

In [16]:
engine = create_engine(conStr)
dbcon = engine.connect()

2. 데이터 조회하기
   - 조회 결과를 pandas의 DataFrame 객체 형태로 반환한다.
   - 조회 속도가 cx_Oracle에 비해 확연히 느림

In [17]:
df = read_sql_table('department', con = dbcon)
df

,deptno,dname,loc
0,101,컴퓨터공학과,1호관
1,102,멀티미디어학과,2호관
2,201,전자공학과,3호관
3,202,컴퓨터공학과,공학관
4,320,정보통신학과,6호관


- 원하는 컬럼만 조회하기

In [18]:
df = read_sql_table('student', columns = ['studno', 'name', 'grade', 'idnum'], con = dbcon)
df

,studno,name,grade,idnum
0,10101,전인하,4,7907021369824
1,10102,박미경,1,8405162123648
2,10103,김영균,3,8103211063421
3,10104,지은경,2,8004122298371
4,10105,임유진,2,8301212196482
5,10106,서재진,1,8511291186273
6,10107,이광훈,4,8109131276431
7,10108,류민정,2,8108192157498
8,10201,김진영,2,8206062186327
9,10202,오유석,4,7709121128379


3. 데이터베이스 접속 해제

In [19]:
dbcon.close()